# Utiliser un LIDAR à INTech

## Analyse du main.py (1er septembre 2018)

### Les imports

On peut séparer les modules importés en 2 catégories : les modules de la bibliothèque standard (les modules apportés quand on installe Python) et les modules que l'on crée soi-même.


Il faut d'abord installer **pyserial** avec :

```python3
pip install pyserial
```

On se place à la racine du paquet.

In [1]:
import os
os.chdir("../..")

De quoi créer un dossier et de vérifier si un fichier est un dossier (il faut se rappeler que tout est fichier dans une arborescence de stockage).

In [2]:
from os import mkdir
from os.path import isdir

Récupération des paramètres définis dans un fichier en .ini

In [3]:
import logging.config

if not isdir("./Logs/"):
    mkdir("./Logs/")


logging.config.fileConfig('./configs/config_log.ini')
_loggerPpl = logging.getLogger("ppl")
_loggerHl = logging.getLogger("hl")

Ecriture dans un fichier CSV (*comma separated values*)

In [4]:
from csv import writer
data_file = open("Logs/RawData.logs", "a")
data_writer = writer(data_file, delimiter=" ")
data_writer.writerow(["#NEW"])

6

*sleep* permet de faire une pause dans un *thread* (ou fil, c'est un moyen de lancer plusieurs opérations en parallèle) et *time* permet de récupérer le nombre de secondes écoulées depuis le 1er janvier 1970.

In [5]:
from time import sleep, time

Gestion de la connexion au code de haut-niveau du robot principal.

In [6]:
from src.HL_connection import hl_connected
from src.HL_connection import hl_socket
from src.HL_connection import stop_com_hl

*ThreadData* est une classe qui hérite de *Thread*. Elle gère la récupération des données provenant du LIDAR.

In [7]:
from src.ThreadData import ThreadData

Usage intensif de **matplotlib** pour afficher les mesures.

In [8]:
from src.affichage import afficher_en_polaire, affichage, affichage_cartesien, affichage_polaire, \
    init_affichage_cartesien, init_affichage_polaire

*mesures* : fonction principale qui récupére et traite les données reçues par le LIDAR. Par exemple, c'est là où on définit les obstacles.

In [9]:
from src.mesures import mesures

Initialisation de variables globales au projet.

In [10]:
socket = None
thread_data = None
ax = None
fig = None
envoi = None

Si le LIDAR est branché, vous pouvez essayer de'éxecuter ce code. Sinon... ça marchera pas !

In [ ]:
try:
    # Liste des positions des anciens obstacles
    list_obstacles_precedente = []

    # Creation de socket pour communiquer avec le HL
    if hl_connected:
        socket = hl_socket()

    # Demarre le Thread recevant les donnees
    thread_data = ThreadData()
    thread_data.start()

    # Attente de quelques tours pour que le lidar prenne sa pleine vitesse et envoie assez de points
    sleep(1)

    # Initialisation de l'affichage
    if not hl_connected and affichage:
        print("Affichage init")
        if afficher_en_polaire:
            ax, fig = init_affichage_polaire()
        else:
            ax, fig = init_affichage_cartesien()

    # Initialisation des valeurs pour le calcul du temps d'exécution
    t = time()
    te = t

    # Boucle de récupération,de traitement des données, d'envoi et d'affichage
    while True:
        # Aucun interet à spammer, on a moins de chance de bloquer l'execution du thread temporairement

        # Calcul du temps d'exécution : aussi utilisé pour le Kalman
        te = (time() - t)
        t = time()
        # On récupère les données du scan du LiDAR et on fait les traitements
        dico, limits, list_obstacles, list_obstacles_precedente = mesures(te, list_obstacles_precedente, thread_data)
        # Envoi de la position du centre de l'obstacle détécté pour traitement par le pathfinding

        liste_envoyee = []
        for o in list_obstacles:
            angle = o.center
            r = dico[angle]
            liste_envoyee.append(str((r, angle)))
            envoi = ";".join(liste_envoyee)
            envoi = envoi + "\n"
        _loggerHl.debug("envoi au hl: %s.", envoi)
        data_writer.writerow(dico.values())
        if hl_connected:
            socket.send(envoi.encode('ascii'))
        thread_data.lidar.clean_input()
        # Affichage des obstacles, de la position Kalman, et des points détectés dans chaque obstacle
        if affichage:
            if afficher_en_polaire:
                affichage_polaire(limits, ax, list_obstacles, dico, fig)
            else:
                affichage_cartesien(limits, ax, list_obstacles, dico, fig)

except KeyboardInterrupt:
    # Arrêt du système
    if hl_connected and socket:
        stop_com_hl(socket)
    _loggerPpl.info("ARRET DEMANDE.")
    if thread_data:
        thread_data.stop_lidar()
        thread_data.join()
        thread_data = None

finally:
    # Arrêt du système
    if hl_connected and socket:
        stop_com_hl(socket)
    if thread_data:
        thread_data.stop_lidar()
        thread_data.join()
        thread_data = None
    data_file.close()